# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 3 2022 8:34AM,247705,10,8604306,"SD Head USA, LLC",Released
1,Oct 3 2022 8:30AM,247684,10,0086036403,ISDIN Corporation,Released
2,Oct 3 2022 8:30AM,247684,10,0086036412,ISDIN Corporation,Released
3,Oct 3 2022 8:30AM,247684,10,0086036426,ISDIN Corporation,Released
4,Oct 3 2022 8:30AM,247684,10,0086036434,ISDIN Corporation,Released
5,Oct 3 2022 8:30AM,247684,10,0086036433,ISDIN Corporation,Released
6,Oct 3 2022 8:30AM,247684,10,0086036493,ISDIN Corporation,Released
7,Oct 3 2022 8:30AM,247684,10,0086036492,ISDIN Corporation,Released
8,Oct 3 2022 8:30AM,247684,10,0086036494,ISDIN Corporation,Released
9,Oct 3 2022 8:30AM,247684,10,0086036501,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
7,247700,Released,8
8,247701,Released,1
9,247702,Released,1
10,247703,Released,1
11,247705,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247700,NaN,8.0
247701,NaN,1.0
247702,NaN,1.0
247703,NaN,1.0
247705,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
247684,0.0,15.0
247687,0.0,36.0
247690,0.0,1.0
247692,0.0,1.0
247696,1.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
247684,0,15
247687,0,36
247690,0,1
247692,0,1
247696,1,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,247684,0,15
1,247687,0,36
2,247690,0,1
3,247692,0,1
4,247696,1,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,247684,,15
1,247687,,36
2,247690,,1
3,247692,,1
4,247696,1,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 3 2022 8:34AM,247705,10,"SD Head USA, LLC"
1,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation
16,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation
52,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.
53,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix
54,Oct 3 2022 8:26AM,247703,19,ACG North America LLC
55,Oct 3 2022 8:25AM,247702,10,"Snap Medical Industries, LLC"
56,Oct 3 2022 8:20AM,247701,16,"SHL Pharma, LLC"
57,Oct 3 2022 8:05AM,247700,10,Emerginnova
65,Oct 3 2022 7:56AM,247697,19,"Innogenix, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 3 2022 8:34AM,247705,10,"SD Head USA, LLC",,1
1,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation,,15
2,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation,,36
3,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.,,1
4,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix,,1
5,Oct 3 2022 8:26AM,247703,19,ACG North America LLC,,1
6,Oct 3 2022 8:25AM,247702,10,"Snap Medical Industries, LLC",,1
7,Oct 3 2022 8:20AM,247701,16,"SHL Pharma, LLC",,1
8,Oct 3 2022 8:05AM,247700,10,Emerginnova,,8
9,Oct 3 2022 7:56AM,247697,19,"Innogenix, LLC",,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 8:34AM,247705,10,"SD Head USA, LLC",1,
1,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation,15,
2,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation,36,
3,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.,1,
4,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix,1,
5,Oct 3 2022 8:26AM,247703,19,ACG North America LLC,1,
6,Oct 3 2022 8:25AM,247702,10,"Snap Medical Industries, LLC",1,
7,Oct 3 2022 8:20AM,247701,16,"SHL Pharma, LLC",1,
8,Oct 3 2022 8:05AM,247700,10,Emerginnova,8,
9,Oct 3 2022 7:56AM,247697,19,"Innogenix, LLC",10,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 8:34AM,247705,10,"SD Head USA, LLC",1,
1,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation,15,
2,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation,36,
3,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.,1,
4,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 8:34AM,247705,10,"SD Head USA, LLC",1,NaN
1,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation,15,NaN
2,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation,36,NaN
3,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.,1,NaN
4,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 8:34AM,247705,10,"SD Head USA, LLC",1,0.0
1,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation,15,0.0
2,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation,36,0.0
3,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.,1,0.0
4,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1733860,63,0.0
16,247701,1,0.0
19,743096,13,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1733860,63,0.0
1,16,247701,1,0.0
2,19,743096,13,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,63,0.0
1,16,1,0.0
2,19,13,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,63.0
1,16,Released,1.0
2,19,Released,13.0
3,10,Executing,0.0
4,16,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19
Status,,,
Executing,0.0,0.0,1.0
Released,63.0,1.0,13.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19
0,Executing,0.0,0.0,1.0
1,Released,63.0,1.0,13.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19
0,Executing,0.0,0.0,1.0
1,Released,63.0,1.0,13.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()